In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [7]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [8]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



In [9]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

# Octal Puzzle Problems

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. An action is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of "inversions"; the other has an odd number. An inversion is when a piece in the state is larger than a piece that follows it.




In [21]:
# board = []
# used = []
# octalSum = []
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)):
        self.initial = initial 
        self.goal = initial
    
    # def actions(self, state):
    #    # The indexes of the squares that the blank can move to.
    #     moves = ((1, 3),    (0, 2, 4),    (1, 5),
    #              (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
    #              (3, 7),    (4, 6, 8),    (7, 5))
    #     blank = state.index(0)
    #     return moves[blank]

     


    def result(self, state, action):
        """I need to replace the values of certain locations with 0. [Swap the blank with the square numbered `action`]."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)



    def is_goal(self, state):
        return super().is_goal(state)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h(self, node): return self.h2(node)

    
    


In [59]:
class OctaPuzzle(Problem):
    def __init__(self, initial, goal = (9,3, 13, 6, 12, 11, 5, 1, 14, 16, 2, 4, 7, 8, 15, 10)):
        #assert inversions(initial) % 2 == inversions(goal)
        self.initial = initial 
        self.goal = goal
    
    def is_goal(self, state):
        (state[1]+state[2]+state[6]+state[12] == 34)and (state[0]+state[8]+state[9]+state[15] == 34) and
        (state[1]+state[4]+state[11]+state[14] == 34)and (state[11]+state[12]+state[13]+state[2] == 34) and
        (state[1]+state[5]+state[9]+state[11] == 34)and (state[4]+state[6]+state[10]+state[14] == 34) and
        (state[0]+state[2]+state[5]+state[7] == 34)and (state[7]+state[9]+state[12]+state[15] == 34) and
        (state[0]+state[3]+state[6]+state[8] == 34) and (state[8]+state[10]+state[13]+state[15] == 34)

        
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h(self, node): return self.h2(node)
    


    def hamming_distance(A, B):
        "Number of positions where vectors A and B are different."
        return sum(a != b for a, b in zip(A, B))
    

    def inversions(board):
        "The number of times a piece is a smaller number than a following piece."
        return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
    def board8(board, fmt=(4 * '{} {} {} {} \n')):
        "A string representing an octal board"
        return fmt.format(*board).replace('_', '0')
        print(board)

   

    # We can swap two number positions that are not equal to 0
    def actions(self, state):
        # We have 16 positions available
        moves = (3,6,5,1)
        
        
        # swap = state.index((0, 3))
        return moves

    def result(self, state, action):
        print(state, self.goal, action)
        # s = list(state)
        # swap = state.index((0, 3))
        # s[action], s[swap] = s[swap], s[action]
        return state

    def print_state(self):
        # Display the state of the board in the octa format 
        str_state = []
        
        # Iterate through all the tiles
        for row in self.state:
            for element in row:
                if element == 0:
                    str_state.append("b")
                else:
                    str_state.append(str(element))
        
        # Print out the resulting state       
        print("".join(str_state[0:3]), "".join(str_state[3:6]), "".join(str_state[6:9]))
    
  
    
    
    

    #     return goal

In [62]:

#OctaPuzzle.get_available_actions( )

In [58]:
o1 = OctaPuzzle((9, 0, 13, 0, 12, 11, 0, 0, 14, 16, 2, 4, 7, 8, 15, 10))


e1 = EightPuzzle((1, 2, 3, 0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16))
# e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
# e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
# e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
# e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

In [41]:

for s in path_states(breadth_first_search(o1)):
    print(s)
   

(9, 0, 13, 0, 12, 11, 0, 0, 14, 16, 2, 4, 7, 8, 15, 10) (9, 3, 13, 6, 12, 11, 5, 1, 14, 16, 2, 4, 7, 8, 15, 10) 3
(9, 0, 13, 0, 12, 11, 0, 0, 14, 16, 2, 4, 7, 8, 15, 10) (9, 3, 13, 6, 12, 11, 5, 1, 14, 16, 2, 4, 7, 8, 15, 10) 6
(9, 0, 13, 0, 12, 11, 0, 0, 14, 16, 2, 4, 7, 8, 15, 10) (9, 3, 13, 6, 12, 11, 5, 1, 14, 16, 2, 4, 7, 8, 15, 10) 5
(9, 0, 13, 0, 12, 11, 0, 0, 14, 16, 2, 4, 7, 8, 15, 10) (9, 3, 13, 6, 12, 11, 5, 1, 14, 16, 2, 4, 7, 8, 15, 10) 1


TypeError: best_first_search() missing 1 required positional argument: 'f'

A*
Solve the eight puzzle using the specified heuristic. The choices for the heuristic are ‘h1’ or ‘h2’. ‘h1’ counts the number of misplaced tiles from the goal state while h2 represents the sum of the Manhattan distances of each tile from its correct position in the goal state. After solving the puzzle, the solution path should be printed as a series of actions to move from the starting state to the goal state. The length of the solution path should also be displayed.

### Conclusion
This project demonstrated solving the octal puzzle using a-star search with two different heuristics. Further work can be carried out on the octa puzzle to involve solving the puzzle using a-star search informed by better sophisticated heuristic like the version implemented by Hanson et al (1992). An additionally intriguing route for exploration of the octa puzzle could be by training a neural network to solve the puzzle using the starting state as a feature and the sequence of actions as a label. Some graduate attempt by Peterson in 2013 utilized neural networks to attempt to derive a better heuristic which I believe could be adopted for the octa puzzle. I was inspired by resources linked to the textbook Artificial Intelligence: A Modern Approach by Russell and Norvig. I thoroughly enjoyed developing the octa puzzle solver using search algorithms and am looking forward to greater challenges in artificial intelligence.